# 10. Double DQN

本节学习 **Double DQN** --- [Deep Reinforcement Learning with Double Q-learning](https://arxiv.org/abs/1509.06461)

同理，先导入以下库：

In [3]:
import random
import numpy as np
from tqdm import tqdm
import torch
import torch.nn.functional as F
import gymnasium as gym

In [4]:
from utils.replay_buffer import ReplayBuffer
from utils.smoothing import moving_average

### 10.1 Double DQN 原理
> 在之前介绍原始DQN算法的 **目标网络** 时，就说到需要注意Q值更新中 **下一状态最优动作的选择** 以及 **对应的最大Q值** 都是由 **目标网络** 计算：
$$r+\gamma Q_{\omega^-}\left(s^{\prime},\arg\max_{a^{\prime}}Q_{\omega^-}\left(s^{\prime},a^{\prime}\right)\right)$$

#### 这里有一个问题: 
神经网络在拟合的过程中本就会导致估算的Q值有正向或负向的误差，又因为每次利用的是下一状态$s^{\prime}$的最大动作Q值，就会导致累积正向误差(当目标网络也更新时):
$$Q_{\mathrm{target}}=r+\gamma\cdot\max_{a^{\prime}}Q(s^{\prime},a^{\prime};\omega^-)$$
使单个目标网络中，某个状态$s_{}$的动作Q值会因为下一个状态$s^{\prime}$的最大Q值正向增大，同时还会将本身Q值的过高估计,影响传递给上一个状态$s_{^-}$

> 对于动作空间较大的任务，DQN 中的过高估计问题会非常严重，造成 DQN 无法有效工作的后果，以下为证明（概率论）：

将估算误差记为：$\epsilon_a=Q_{\omega^-}(s,a)-\max_{a^{\prime}}Q^*(s,a^{\prime})$ ，动作有m个
又估算误差对于不同的动作是独立的：$P\left(\max_a\epsilon_a\leq x\right)=\prod_{a=1}^mP\left(\epsilon_a\leq x\right)$
为简化实际环境，且已知估算误差越小概率越小，所以假设估算误差服从$[-1,1]$的均匀分布，此时有：
$$\begin{aligned}
P\left(\max_{a}\epsilon_{a}\leq x\right) & =\prod_{a=1}^mP(\epsilon_a\leq x) \\
 & =
\begin{cases}
0 & \mathrm{if}x\leq-1 \\
\left(\frac{1+x}{2}\right)^m & \mathrm{if}x\in(-1,1) \\
1 & \mathrm{if}x\geq1 & 
\end{cases}
\end{aligned}$$
从而得到最大估算误差的期望为：
$$\begin{aligned}
\mathbb{E}\left[\max_{a}\epsilon_{a}\right] & =\int_{-1}^{1}x\frac{\mathrm{d}}{\mathrm{d}x}P\left(\max_{a}\epsilon_{a}\leq x\right)\mathrm{d}x \\
 & =\left[\left(\frac{x+1}{2}\right)^m\frac{mx-1}{m+1}\right]_{-1}^1 \\
 & =\frac{m-1}{m+1}
\end{aligned}$$
- m越大误差越大

#### Double DQN 的解决方案：
为解决这一问题，Double DQN 算法提出利用两个独立训练的神经网络估算：
$$r+\gamma Q_{\omega^-}\left(s^{\prime},\arg\max_{a^{\prime}}Q_\omega\left(s^{\prime},a^{\prime}\right)\right)$$
此时传统 DQN 算法当中的训练网络，负责起对下个状态所采取的动作进行选择
> 采用 **动作的选择** 与 **Q值的评估** 分离的方式，可以在一定程度上减少“更新滞后的目标网络愈发高估之前作出的错误的动作”的现象，通过评估实时更新的训练网络所选择的动作，增添新的可能，两网络共同作用，最终可以显著减少 Q 值估计的高估问题

### 10.2 Double DQN 代码实现


#### Q网络：

In [ ]:
class Qnet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(Qnet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

#### Double DQN算法：

In [ ]:
class DoubleDQN:
    def __init__(self, state_dim, hidden_dim, action_dim, learning_rate, gamma, epsilon, target_update, device):
        self.action_dim = action_dim 
        # Q 网络
        self.q_net = Qnet(state_dim, hidden_dim, self.action_dim).to(device) 
        # 目标网络
        self.target_q_net = Qnet(state_dim, hidden_dim, self.action_dim).to(device)
        # 使用Adam优化器
        self.optimizer = torch.optim.Adam(self.q_net.parameters(), lr=learning_rate)
        self.gamma = gamma  # 折扣因子（未来）
        self.epsilon = epsilon  # epsilon-贪婪策略
        self.target_update = target_update  # 目标网络更新频率
        self.count = 0  # 计数器,记录更新次数
        self.device = device

    def take_action(self, state):  # epsilon-贪婪策略采取动作
        if np.random.random() < self.epsilon:
            action = np.random.randint(self.action_dim)
        else:
            # 将列表转换为(batch_size, state_dim)的结构与网络传播兼容
            state = torch.tensor(np.array([state]), dtype=torch.float).to(self.device)
            action = self.q_net(state).argmax().item()
        return action
    
    def max_q_value(self, state):
        state = torch.tensor(np.array([state]), dtype=torch.float).to(self.device)
        return self.q_net(state).max().item()

    def update(self, transition_dict):
        # .view(-1, 1)转换为一个2D张量，以便与其他维度匹配并符合模型的输入要求
        states = torch.tensor(transition_dict['states'], dtype=torch.float).to(self.device)
        actions = torch.tensor(transition_dict['actions']).view(-1, 1).to(self.device)#转换为一个2D张量，以便与其他维度匹配并符合模型的输入要求
        rewards = torch.tensor(transition_dict['rewards'], dtype=torch.float).view(-1, 1).to(self.device)
        next_states = torch.tensor(transition_dict['next_states'], dtype=torch.float).to(self.device)
        dones = torch.tensor(transition_dict['dones'], dtype=torch.float).view(-1, 1).to(self.device)

        q_values = self.q_net(states).gather(1, actions)  # 当前Q值
        
        #---------------------------------------------
        max_action = self.q_net(next_states).max(1)[1].view(-1, 1)
        max_next_q_values = self.target_q_net(next_states).gather(1, max_action)
        #max_next_q_values = self.target_q_net(next_states).max(1)[0].view(-1, 1)  # DQN的情况
        #----------------------------------------------
        
        q_targets = rewards + self.gamma * max_next_q_values * (1 - dones)  # TD误差目标
        # dqn_loss = torch.mean(F.mse_loss(q_values, q_targets))  # 均方误差损失函数
        loss = F.mse_loss(q_values, q_targets)  # 默认已经是 mean
        self.optimizer.zero_grad()  # PyTorch中默认梯度会累积, 非批量累积梯度任务，这里需要显式将梯度置为0
        loss.backward()  # 反向传播更新参数
        self.optimizer.step()

        if self.count % self.target_update == 0:
            self.target_q_net.load_state_dict(
                self.q_net.state_dict())  # 更新目标网络
        self.count += 1

#### Env设置: